<a href="https://colab.research.google.com/github/FelixLeChat92/MyPrompts/blob/main/DeepSeek_R1_Distill_Qwen_7B_unsloth_FineTune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title 1. Setup and Dependencies
%%capture
!pip install -U bitsandbytes
!pip install -U transformers==4.39.2
!pip install -U peft==0.9.0
!pip install -U accelerate==0.28.0
!pip install -U datasets==2.17.0
!pip install -U trl==0.7.10
!pip install -U einops
!pip uninstall -y protobuf
!pip install --no-binary :all: protobuf
!pip install -U unsloth==2024.4
!pip install -U wandb

import os
import sys

BASE_PATH = '/content'
PROJECT_PATH = os.path.join(BASE_PATH, 'LLaMA-Factory')
os.chdir(BASE_PATH)

# Clone the repository with depth 1 for faster cloning
if not os.path.exists(PROJECT_PATH):
    !git clone --depth 1 https://github.com/hiyouga/LLaMA-Factory.git {PROJECT_PATH}

# Move to the project directory
os.chdir(PROJECT_PATH)

# Install LLaMA Factory
!pip install -e .[torch,bitsandbytes]

In [ ]:
#@title 2. Check GPU Environment
import torch

try:
    assert torch.cuda.is_available() is True
    print("GPU is available. Proceeding with setup.")
except AssertionError:
    print("Error: Please set up a GPU before using LLaMA Factory.")
    print("Instructions for setting up a GPU in Colab: https://medium.com/mlearning-ai/training-yolov4-on-google-colab-316f8fff99c6")
    sys.exit(1)

In [ ]:
#@title 3. Create Medical Dataset Configuration
%%writefile {PROJECT_PATH}/data/dataset_info.json
{
    "medical_qa": {
        "hf_hub_url": "keivalya/MedQuad-MedicalQnADataset",
        "columns": {
            "prompt": "Question",
            "response": "Answer"
        }
    }
}

In [ ]:
#@title 4. Define Parameters for Model Fine-Tuning
model_name = "unsloth/DeepSeek-R1-Distill-Qwen-7B-unsloth-bnb-4bit" #@param {type:"string"}
dataset = "medical_qa" #@param {type:"string"}
template = "qwen" #@param {type:"string"}
finetuning_type = "lora" #@param {type:"string"}
lora_target = "all" #@param {type:"string"}
output_dir = "deepseek_lora" #@param {type:"string"}
per_device_train_batch_size = 1 #@param {type:"integer"}
gradient_accumulation_steps = 8 #@param {type:"integer"}
lr_scheduler_type = "cosine" #@param {type:"string"}
logging_steps = 1 #@param {type:"integer"}
warmup_ratio = 0.1 #@param {type:"number"}
save_steps = 10 #@param {type:"integer"}
learning_rate = 5e-5 #@param {type:"number"}
num_train_epochs = 3.0 #@param {type:"number"}
max_samples = 500 #@param {type:"integer"}
max_grad_norm = 0.3 #@param {type:"number"}
loraplus_lr_ratio = 16.0 #@param {type:"number"}
fp16 = True #@param {type:"boolean"}
report_to = "wandb" #@param {type:"string"}
run_name = "sft-deepseek-medical-qa" #@param {type:"string"}

In [ ]:
#@title 5. Fine-tune Model via Command Line
from google.colab import userdata
import os

# Initialize environment variable for W&B API key from Colab userdata
os.environ["WANDB_API_KEY"] = userdata.get("WANDB_API_KEY")
os.environ["WANDB_PROJECT"] = "llama-factory-finetune"

!CUDA_VISIBLE_DEVICES=0 llamafactory-cli train \
    --stage sft \
    --do_train \
    --model_name_or_path {model_name} \
    --dataset {dataset} \
    --template {template} \
    --finetuning_type {finetuning_type} \
    --lora_target {lora_target} \
    --output_dir {output_dir} \
    --per_device_train_batch_size {per_device_train_batch_size} \
    --gradient_accumulation_steps {gradient_accumulation_steps} \
    --lr_scheduler_type {lr_scheduler_type} \
    --logging_steps {logging_steps} \
    --warmup_ratio {warmup_ratio} \
    --save_steps {save_steps} \
    --learning_rate {learning_rate} \
    --num_train_epochs {num_train_epochs} \
    --max_samples {max_samples} \
    --max_grad_norm {max_grad_norm} \
    --loraplus_lr_ratio {loraplus_lr_ratio} \
    --fp16 {fp16} \
    --report_to {report_to} \
    --run_name {run_name}

In [ ]:
#@title 6. Merge the LoRA Adapter (Optional)
merge = True #@param {type:"boolean"}
if merge:
    !llamafactory-cli export \
        --model_name_or_path "unsloth/DeepSeek-R1-Distill-Qwen-7B" \
        --adapter_name_or_path {output_dir} \
        --template {template} \
        --finetuning_type {finetuning_type} \
        --export_dir deepseek_lora_merged \
        --export_size 2 \
        --export_device "cpu"
    print("LoRA adapter merged successfully. Check the 'deepseek_lora_merged' directory.")
else:
    print("LoRA adapter merge skipped.")

In [ ]:
#@title 7. Download the Merged Model (Optional)
download_merged = True #@param {type:"boolean"}
if download_merged:
    from google.colab import files
    !zip -r deepseek_lora_merged.zip deepseek_lora_merged
    files.download('deepseek_lora_merged.zip')
else:
    print("Merged model download skipped.")

In [ ]:
from huggingface_hub import login, create_repo, upload_folder
from google.colab import userdata
HF_TOKEN=userdata.get('HF_TOKEN')

# Log in to Hugging Face (replace with your token)
login(token=HF_TOKEN)

# Create a repository (if it doesn't exist)
repo_name = "Felprot75/deepseek-lora-merged"  # Replace with desired repo name
create_repo(repo_name, exist_ok=True)

# Upload the model folder
upload_folder(
    repo_id=repo_name,
    folder_path="deepseek_lora_merged",
    path_in_repo="",  # Upload to root of repo
    commit_message="Upload merged DeepSeek LoRA model",
)

print(f"Model uploaded to Hugging Face: {repo_name}")